### Матричные факторизации

В данной работе вам предстоит познакомиться с практической стороной матричных разложений.
Работа поделена на 4 задания:
1. Вам необходимо реализовать SVD разложения используя SGD на explicit данных
2. Вам необходимо реализовать матричное разложения используя ALS на implicit данных
3. Вам необходимо реализовать матричное разложения используя BPR(pair-wise loss) на implicit данных
4. Вам необходимо реализовать матричное разложения используя WARP(list-wise loss) на implicit данных

Мягкий дедлайн 28 Сентября (пишутся замечания, выставляется оценка, есть возможность исправить до жесткого дедлайна)

Жесткий дедлайн 5 Октября (Итоговая проверка)

In [30]:
import implicit
import pandas as pd
import numpy as np
import scipy.sparse as sp

from scipy.sparse import random
from scipy import stats
from tqdm.notebook import tqdm
from sklearn.metrics import mean_squared_error
import scipy.sparse

from scipy.sparse.linalg import spsolve

from tqdm.notebook import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity

В данной работе мы будем работать с explicit датасетом movieLens, в котором представленны пары user_id movie_id и rating выставленный пользователем фильму

Скачать датасет можно по ссылке https://grouplens.org/datasets/movielens/1m/

In [14]:
ratings = pd.read_csv('ml-1m/ratings.dat', delimiter='::', header=None, 
        names=['user_id', 'movie_id', 'rating', 'timestamp'], 
        usecols=['user_id', 'movie_id', 'rating'], engine='python')

In [15]:
movie_info = pd.read_csv('ml-1m/movies.dat', delimiter='::', header=None, 
        names=['movie_id', 'name', 'category'], engine='python')

Explicit данные

In [16]:
ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


Для того, чтобы преобразовать текущий датасет в Implicit, давайте считать что позитивная оценка это оценка >=4

In [17]:
implicit_ratings = ratings.loc[(ratings['rating'] >= 4)]

In [18]:
implicit_ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
3,1,3408,4
4,1,2355,5
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4
10,1,595,5
11,1,938,4
12,1,2398,4


Удобнее работать с sparse матричками, давайте преобразуем DataFrame в CSR матрицы

In [19]:
users = implicit_ratings["user_id"]
movies = implicit_ratings["movie_id"]
user_item = sp.coo_matrix((np.ones_like(users), (users, movies)))
user_item_t_csr = user_item.T.tocsr()
user_item_csr = user_item.tocsr()

В качестве примера воспользуемся ALS разложением из библиотеки implicit

Зададим размерность латентного пространства равным 64, это же определяет размер user/item эмбедингов

In [20]:
model = implicit.als.AlternatingLeastSquares(factors=64, iterations=100, calculate_training_loss=True)

В качестве loss здесь всеми любимый RMSE

In [21]:
model.fit(user_item_t_csr)

KeyboardInterrupt: 

Построим похожие фильмы по 1 movie_id = Истории игрушек

In [ ]:
movie_info.head(5)

In [ ]:
get_similars = lambda item_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                        for x in model.similar_items(item_id)]

Как мы видим, симилары действительно оказались симиларами.

Качество симиларов часто является хорошим способом проверить качество алгоритмов.

P.S. Если хочется поглубже разобраться в том как разные алгоритмы формируют разные латентные пространства, рекомендую загружать полученные вектора в tensorBoard и смотреть на сформированное пространство

In [ ]:
get_similars(1, model)

Давайте теперь построим рекомендации для юзеров

Как мы видим юзеру нравится фантастика, значит и в рекомендациях ожидаем увидеть фантастику

In [ ]:
get_user_history = lambda user_id, implicit_ratings : [movie_info[movie_info["movie_id"] == x]["name"].to_string() 
                                            for x in implicit_ratings[implicit_ratings["user_id"] == user_id]["movie_id"]]

In [ ]:
get_user_history(4, implicit_ratings)

Получилось! 

Мы действительно порекомендовали пользователю фантастику и боевики, более того встречаются продолжения тех фильмов, которые он высоко оценил

In [ ]:
get_recommendations = lambda user_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                               for x in model.recommend(user_id, user_item_csr)]

In [ ]:
get_recommendations(4, model)

Теперь ваша очередь реализовать самые популярные алгоритмы матричных разложений

Что будет оцениваться:
1. Корректность алгоритма
2. Качество получившихся симиларов
3. Качество итоговых рекомендаций для юзера

### Задание 1. Не использую готовые решения, реализовать SVD разложение используя SGD на explicit данных

In [42]:
users = ratings["user_id"]
movies = ratings["movie_id"]
user_item = sp.coo_matrix((ratings["rating"], (users, movies)))

In [43]:
class SGD:
    def __init__(self, lr=1e-2, lamb=1e-2, feat_num=64, num_epochs=10):
        self.lr = lr
        self.lamb = lamb
        self.feat_num = feat_num
        self.num_epochs = num_epochs
        
    def fit(self, user_item, ratings):
        W = np.random.uniform(0, 1/np.sqrt(self.feat_num), (user_item.shape[0], self.feat_num))
        H = np.random.uniform(0, 1/np.sqrt(self.feat_num), (self.feat_num, user_item.shape[1]))
        
        B_w = np.random.normal(0, 1, (user_item.shape[0]))
        B_h = np.random.normal(0, 1, (user_item.shape[1]))

        gen_bias = np.mean(ratings)
        for _ in tqdm(range(self.num_epochs)):
            for i, j, v in zip(tqdm(user_item.row, leave=False,), user_item.col, user_item.data):
                error = (W[i, :] @ H[:, j] + B_w[i] +  B_h[j] + gen_bias) - v
                W_i_old = W[i, :].copy()
                B_w[i] -= self.lr * (error + self.lamb * B_w[i])
                B_h[j] -= self.lr * (error + self.lamb * B_h[j])
                W[i, :] -= self.lr * (error * H[:, j].T + self.lamb * (W[i, :] + B_w[i]))
                H[:, j] -= self.lr * (error * W_i_old.T + self.lamb * (H[:, j] + B_h[j]))
            print(mean_squared_error((W @ H + B_w.reshape(-1, 1) + B_h.reshape(1, -1) + gen_bias)[user_item.row, user_item.col], user_item.data))
        return W, H

In [44]:
sg = SGD()

In [45]:
W, H = sg.fit(user_item, ratings['rating'].to_numpy())

0.9033509984527467
0.8394743052590462
0.8162359480538356
0.7865131675842894
0.7509971735171391
0.7138552783141323
0.6760181283360066
0.6396646749991335
0.6066048253592881
0.5779535962584311



In [46]:
get_similars = lambda item_id, sim : [movie_info[movie_info["movie_id"] == x]["name"].to_string() 
                                        for x in reversed(np.argsort(sim[item_id,:])[-10:])]

get_similars(1, cosine_similarity(H.T))

['0    Toy Story (1995)',
 '3045    Toy Story 2 (1999)',
 '584    Aladdin (1992)',
 '591    Beauty and the Beast (1991)',
 "2286    Bug's Life, A (1998)",
 '1838    Mulan (1998)',
 '1526    Hercules (1997)',
 '148    Apollo 13 (1995)',
 '2021    Rescuers, The (1977)',
 '2618    Tarzan (1999)']

In [56]:
get_similars = lambda item_id, sim : [movie_info[movie_info["movie_id"] == x]["name"].to_string() 
                                        for x in reversed(np.argsort(sim[item_id,:])[-10:])]

get_similars(1982, cosine_similarity(H.T))

['1913    Halloween (1978)',
 '956    Night of the Living Dead (1968)',
 '1326    Nightmare on Elm Street, A (1984)',
 '2386    Fly, The (1986)',
 '1114    Howling, The (1980)',
 '2565    Mummy, The (1959)',
 '1301    American Werewolf in London, An (1981)',
 '2481    Haunting, The (1963)',
 '2390    Texas Chainsaw Massacre, The (1974)',
 '1324    Carrie (1976)']

In [48]:
get_similars = lambda item_id, sim : [movie_info[movie_info["movie_id"] == x]["name"].to_string() 
                                        for x in reversed(np.argsort(sim[item_id,:])[-10:])]

get_similars(2628, cosine_similarity(H.T))

['2559    Star Wars: Episode I - The Phantom Menace (1999)',
 '1192    Star Wars: Episode VI - Return of the Jedi (1983)',
 '1178    Star Wars: Episode V - The Empire Strikes Back...',
 '257    Star Wars: Episode IV - A New Hope (1977)',
 '2191    Wisdom (1986)',
 '325    Star Trek: Generations (1994)',
 '3013    World Is Not Enough, The (1999)',
 '1335    Star Trek: First Contact (1996)',
 '1358    Young Guns II (1990)',
 '2324    Star Trek: Insurrection (1998)']

### Задание 2. Не использую готовые решения, реализовать матричное разложение используя ALS на implicit данных

In [ ]:
users = implicit_ratings["user_id"]
movies = implicit_ratings["movie_id"]
user_item = sp.coo_matrix((np.ones_like(users), (users, movies)))
user_item_t_csr = user_item.T.tocsr()
user_item_csr = user_item.tocsr()


In [34]:
class ALS:
    def __init__(self, lamb, n_iters, n_factors, alpha):
        self.lamb = lamb
        self.alpha = alpha
        self.n_iters = n_iters
        self.n_factors = n_factors
    
    def fit(self, ratings):
        #У нас спарс матрицы - единички просто так не прибавить, попробуем это сделать в другом месте
        Cui = ratings.copy().tocsr()
        Cui.data *= self.alpha
        Ciu = Cui.T.tocsr()
        self.n_users, self.n_items = Cui.shape

        rstate = np.random.RandomState(228)
        self.W = scipy.sparse.csr_matrix(rstate.normal(size = (self.n_users, self.n_factors)))
        self.H = scipy.sparse.csr_matrix(rstate.normal(size = (self.n_items, self.n_factors)))
        
        for _ in trange(self.n_iters, desc = 'training'):
            self._als_step(Cui, self.W, self.H, self.n_users)
            self._als_step(Ciu, self.H, self.W, self.n_items)
        
        return self
    
    def _als_step(self, Cui, X, Y, n_shag):
        YtY = Y.T.dot(Y)
        Y_eye = scipy.sparse.eye(Y.shape[0])
        lambda_eye = self.lamb * scipy.sparse.eye(self.n_factors)
        for u in tqdm(range(n_shag)):
            conf_samp = Cui[u,:].toarray()
            pref = conf_samp.copy()
            pref[pref != 0] = 1
            cui_loc = scipy.sparse.diags(conf_samp, [0])
            A = YtY + Y.T.dot(cui_loc).dot(Y)+lambda_eye
            b = Y.T.dot(cui_loc+Y_eye).dot(pref.T)
            
            X[u] = spsolve(A, b)

        return self
als = ALS(n_iters = 15, n_factors = 64, alpha = 15, lamb = 0.01)
als.fit(user_item_csr)

In [37]:
get_similars = lambda item_id, sim : [movie_info[movie_info["movie_id"] == x]["name"].to_string()
                                        for x in reversed(np.argsort(sim[item_id,:])[-10:])]

get_similars(1, cosine_similarity(als.H))

['0    Toy Story (1995)',
 '3045    Toy Story 2 (1999)',
 "2286    Bug's Life, A (1998)",
 '1245    Groundhog Day (1993)',
 '33    Babe (1995)',
 '584    Aladdin (1992)',
 '2327    Shakespeare in Love (1998)',
 '2252    Pleasantville (1998)',
 '352    Forrest Gump (1994)',
 "1854    There's Something About Mary (1998)"]

In [39]:

get_similars = lambda item_id, sim : [movie_info[movie_info["movie_id"] == x]["name"].to_string()
                                        for x in reversed(np.argsort(sim[item_id,:])[-10:])]

get_similars(2628, cosine_similarity(als.H))

['2559    Star Wars: Episode I - The Phantom Menace (1999)',
 '476    Jurassic Park (1993)',
 '1539    Men in Black (1997)',
 '1192    Star Wars: Episode VI - Return of the Jedi (1983)',
 '585    Terminator 2: Judgment Day (1991)',
 '257    Star Wars: Episode IV - A New Hope (1977)',
 '770    Independence Day (ID4) (1996)',
 '2502    Matrix, The (1999)',
 '1178    Star Wars: Episode V - The Empire Strikes Back...',
 '2847    Total Recall (1990)']

### Задание 3. Не использую готовые решения, реализовать матричное разложение BPR на implicit данных


### Задание 4. Не использую готовые решения, реализовать матричное разложение WARP на implicit данных